#### Problem Statement

Banks invest significant resources into outbound marketing campaigns, like calling customers to offer term deposit products. However, these campaigns can be inefficient, contacting many customers who are not interested, leading to:

- High operational costs

- Low conversion rates

- Customer fatigue and churn risk

1. This project seeks to optimize marketing effectiveness by:

2. Segmenting customers based on their financial and demographic behavior

3. Predicting which customers are most likely to subscribe to a term deposit

4. Designing a targeting strategy that reduces wasted outreach while maximizing ROI


**Stakeholders**

|Stakeholder              | What they care About
|-------------------------|------------------------
|Marketing Team           |Improve campaign efficiency and ROI; avoid over-contacting customers|
Data Science Team         |Build interpretable, reliable models with business impact
Compliance/Legal          |Ensure targeting practices are fair, non-discriminatory
Sales / Call Center       |Focus effort on the right customers; increase success rates
Senior Management         |Strategic insights on customer behavior and product interest


#### Imports

Import the required libraries

In [1]:
import pandas as pd
import zipfile
import urllib.request
import io
import os
from google.cloud import bigquery


### Loading dataset

Dataset is from Google [Big Query](https://console.cloud.google.com/bigquery?ws=!1m4!1m3!3m2!1sbigquery-public-data!2sthelook_ecommerce).
A function was writen to obtain the tables in the dataset. The ecommerce data has `7` tables namely:  "distribution_centers", "events", "inventory_items", "order_items", "orders", "products", "users".

In [2]:

client = bigquery.Client(project="testing-469722")

def load_thelook_tables(dataset_id="bigquery-public-data.thelook_ecommerce", table_names=None):
    """
    Load multiple tables from thelook_ecommerce dataset into separate DataFrames.

    Parameters:
        dataset_id (str): Full BigQuery dataset path
        table_names (list): List of table names to load

    Returns:
        dict: Dictionary of {table_name: DataFrame}
    """
    if table_names is None:
        table_names = [
            "distribution_centers",
            "events",
            "inventory_items",
            "order_items",
            "orders",
            "products",
            "users"
        ]

    dataframes = {}

    for table in table_names:
        query = f"SELECT * FROM `{dataset_id}.{table}`"
        df = client.query(query).to_dataframe()
        dataframes[table] = df
        print(f"Loaded {table}: {df.shape[0]} rows")

    return dataframes

### Load the dataset

The dataset are pulled from Big query.


In [3]:
thelook_dfs = load_thelook_tables()

C:\Users\DELL XPS\virtual\dsb_602\Lib\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


Loaded distribution_centers: 10 rows
Loaded events: 2429240 rows
Loaded inventory_items: 490519 rows
Loaded order_items: 181597 rows
Loaded orders: 125188 rows
Loaded products: 29120 rows
Loaded users: 100000 rows


### Accessing the tables


In [4]:
# order table

orders_df = thelook_dfs['orders']
orders_df.head(4)

,order_id,user_id,status,gender,created_at,returned_at,shipped_at,delivered_at,num_of_item
0,8,5,Cancelled,F,2025-03-04 06:46:00+00:00,NaT,NaT,NaT,1
1,9,6,Cancelled,F,2025-06-03 03:06:00+00:00,NaT,NaT,NaT,1
2,16,15,Cancelled,F,2022-04-04 11:48:00+00:00,NaT,NaT,NaT,1
3,44,39,Cancelled,F,2025-08-11 05:15:00+00:00,NaT,NaT,NaT,1


**orders_df table dictionary**

| Column Name    | Description                                                           |
| -------------- | --------------------------------------------------------------------- |
| `order_id`     | Unique identifier for each order                                      |
| `user_id`      | Foreign key that links to the customer (`users.id`)                   |
| `status`       | Status of the order (e.g., `Complete`, `Returned`, `Cancelled`)       |
| `gender`       | Gender of the customer (from the `users` table, if joined)            |
| `created_at`   | Timestamp when the order was placed                                   |
| `returned_at`  | Timestamp when the item was returned (null if not returned)           |
| `shipped_at`   | Timestamp when the item was shipped                                   |
| `delivered_at` | Timestamp when the item was delivered to the customer                 |
| `num_of_item`  | Number of individual items in that order (usually from `order_items`) |


In [5]:
# distribution_centers

distribution_df = thelook_dfs['distribution_centers']
distribution_df.head(5)

,id,name,latitude,longitude,distribution_center_geom
0,8,Mobile AL,30.6944,-88.0431,POINT(-88.0431 30.6944)
1,9,Charleston SC,32.7833,-79.9333,POINT(-79.9333 32.7833)
2,6,Port Authority of New York/New Jersey NY/NJ,40.6340,-73.7834,POINT(-73.7834 40.634)
3,5,New Orleans LA,29.9500,-90.0667,POINT(-90.0667 29.95)
4,2,Chicago IL,41.8369,-87.6847,POINT(-87.6847 41.8369)


**distribution_df dictionary**

| Column Name                | Description                                                                 |
| -------------------------- | --------------------------------------------------------------------------- |
| `id`                       | Unique identifier for each distribution center                              |
| `name`                     | City and state abbreviation of the distribution center location             |
| `latitude`                 | Latitude coordinate of the distribution center (for mapping or distance)    |
| `longitude`                | Longitude coordinate of the distribution center                             |
| `distribution_center_geom` | Geometry field representing the center as a geographic point (GeoJSON-like) |


In [6]:
events_df = thelook_dfs['events']
events_df.head(5)

,id,user_id,sequence_number,session_id,created_at,ip_address,city,state,postal_code,browser,traffic_source,uri,event_type
0,2252100,<NA>,3,e3fc7136-1c43-43ab-9de7-ce2ce5caff62,2022-06-27 11:57:00+00:00,16.139.234.95,São Paulo,São Paulo,02675-031,Firefox,Organic,/cancel,cancel
1,2285223,<NA>,3,903f30d8-4491-471c-bd44-5fc65375b069,2021-10-22 19:03:00+00:00,3.205.142.27,São Paulo,São Paulo,02675-031,IE,Facebook,/cancel,cancel
2,2217907,<NA>,3,1501e6b7-dbf7-4d4c-93e4-5c36e6c7b7af,2021-08-28 04:03:00+00:00,182.41.211.27,São Paulo,São Paulo,02675-031,Other,Email,/cancel,cancel
3,1517876,<NA>,3,f6072efc-f9e5-4a60-8e06-b4a7a8ecbf26,2020-10-24 09:25:00+00:00,71.67.3.226,São Paulo,São Paulo,04941-175,Firefox,Organic,/cancel,cancel
4,1754892,<NA>,3,3ca48bf1-822d-4bbf-87c4-4e0b91ad7878,2024-11-26 07:25:00+00:00,79.186.77.31,Embu-Guaçu,São Paulo,06900-000,Safari,Email,/cancel,cancel


**events_df dictionary**

| Column Name       | Description                                                           |
| ----------------- | --------------------------------------------------------------------- |
| `id`              | Unique identifier for each event record                               |
| `user_id`         | ID of the user associated with the event *(can be missing/anonymous)* |
| `sequence_number` | Order of the event within a session (helps track user journey)        |
| `session_id`      | Unique ID for the user session (used to group related events)         |
| `created_at`      | Timestamp when the event occurred                                     |
| `ip_address`      | User’s IP address at the time of the event                            |
| `city`            | City of the user (inferred from IP)                                   |
| `state`           | State of the user (inferred from IP)                                  |
| `postal_code`     | Postal code of the user (inferred from IP)                            |
| `browser`         | Browser used by the user (e.g., Chrome, Safari)                       |
| `traffic_source`  | Source that led the user to the site (e.g., Organic, Email, Adwords)  |
| `uri`             | Specific page or endpoint visited (e.g., `/cancel`)                   |
| `event_type`      | Type of user action or event (e.g., `cancel`, `purchase`, `checkout`) |


In [97]:
inventoryitems_df = thelook_dfs['inventory_items']
inventoryitems_df.head()

,id,product_id,created_at,sold_at,cost,product_category,product_name,product_brand,product_retail_price,product_department,product_sku,product_distribution_center_id
0,90209,4213,2022-04-30 10:32:09+00:00,2022-05-20 02:33:09+00:00,16.24435,Jumpsuits & Rompers,A Elan Usa Beautiful Strapless Jumpsuit w/ 3/4...,Elan USA,32.950001,Women,872DD316EB8A432CBC63F141E2D68DED,2
1,90210,4213,2021-10-01 07:20:00+00:00,NaT,16.24435,Jumpsuits & Rompers,A Elan Usa Beautiful Strapless Jumpsuit w/ 3/4...,Elan USA,32.950001,Women,872DD316EB8A432CBC63F141E2D68DED,2
2,90211,4213,2020-03-28 03:46:00+00:00,NaT,16.24435,Jumpsuits & Rompers,A Elan Usa Beautiful Strapless Jumpsuit w/ 3/4...,Elan USA,32.950001,Women,872DD316EB8A432CBC63F141E2D68DED,2
3,90212,4213,2022-04-04 08:01:00+00:00,NaT,16.24435,Jumpsuits & Rompers,A Elan Usa Beautiful Strapless Jumpsuit w/ 3/4...,Elan USA,32.950001,Women,872DD316EB8A432CBC63F141E2D68DED,2
4,146617,4213,2024-12-31 11:56:22+00:00,2025-01-12 11:32:22+00:00,16.24435,Jumpsuits & Rompers,A Elan Usa Beautiful Strapless Jumpsuit w/ 3/4...,Elan USA,32.950001,Women,872DD316EB8A432CBC63F141E2D68DED,2


**inventoryitems_df dictionary**

| Column Name                      | Description                                                                                                 |
| -------------------------------- | ----------------------------------------------------------------------------------------------------------- |
| `id`                             | Unique identifier for each inventory record                                                                 |
| `product_id`                     | ID of the product listed in the inventory                                                                   |
| `created_at`                     | Timestamp when the product was added to inventory                                                           |
| `sold_at`                        | Timestamp when the product was sold (null if unsold)                                                        |
| `cost`                           | Cost to the business for this unit (wholesale price)                                                        |
| `product_category`               | Category label for the product (e.g., *Jumpsuits & Rompers*)                                                |
| `product_name`                   | Full name or title of the product                                                                           |
| `product_brand`                  | Brand/manufacturer of the product                                                                           |
| `product_retail_price`           | Suggested retail price or sale price to customer                                                            |
| `product_department`             | Department the product belongs to (e.g., *Women*)                                                           |
| `product_sku`                    | Unique stock keeping unit (SKU) for inventory tracking                                                      |
| `product_distribution_center_id` | Foreign key linking to the distribution center where the item is stocked (join with `distribution_centers`) |


In [8]:
orderitems_df = thelook_dfs['order_items']
orderitems_df.head(5)

,id,order_id,user_id,product_id,inventory_item_id,status,created_at,shipped_at,delivered_at,returned_at,sale_price
0,111996,77190,61515,14235,302621,Complete,2023-05-24 20:52:01+00:00,2023-05-26 10:55:00+00:00,2023-05-29 12:40:00+00:00,NaT,0.02
1,46135,31723,25296,14235,124693,Processing,2025-08-07 04:48:18+00:00,NaT,NaT,NaT,0.02
2,51295,35248,28132,14235,138612,Returned,2024-02-26 09:48:25+00:00,2024-02-28 17:19:00+00:00,2024-03-03 03:36:00+00:00,2024-03-04 06:32:00+00:00,0.02
3,51986,35737,28544,14235,140490,Shipped,2023-06-15 12:07:34+00:00,2023-06-18 06:34:00+00:00,NaT,NaT,0.02
4,95388,65821,52487,14235,257767,Shipped,2025-02-12 00:52:53+00:00,2025-02-11 04:32:00+00:00,NaT,NaT,0.02


**orderitems_df**

| Column Name         | Description                                                                    |
| ------------------- | ------------------------------------------------------------------------------ |
| `id`                | Unique identifier for each order item (line item)                              |
| `order_id`          | Foreign key referencing the `orders` table (represents the main order)         |
| `user_id`           | ID of the customer who placed the order                                        |
| `product_id`        | ID of the product ordered (from `products` or `inventories` table)             |
| `inventory_item_id` | Foreign key linking to the specific item in the inventory (from `inventories`) |
| `status`            | Status of the item within the order: `Complete`, `Cancelled`, etc.             |
| `created_at`        | Timestamp when the item was added to the order                                 |
| `shipped_at`        | Timestamp when the item was shipped                                            |
| `delivered_at`      | Timestamp when the item was delivered                                          |
| `returned_at`       | Timestamp if the item was returned                                             |
| `sale_price`        | Price at which the item was sold to the customer                               |


In [9]:
products_df = thelook_dfs['products']
products_df.head(5)

,id,cost,category,name,brand,retail_price,department,sku,distribution_center_id
0,13842,2.51875,Accessories,Low Profile Dyed Cotton Twill Cap - Navy W39S55D,MG,6.25,Women,EBD58B8A3F1D72F4206201DA62FB1204,1
1,13928,2.33835,Accessories,Low Profile Dyed Cotton Twill Cap - Putty W39S55D,MG,5.95,Women,2EAC42424D12436BDD6A5B8A88480CC3,1
2,14115,4.87956,Accessories,Enzyme Regular Solid Army Caps-Black W35S45D,MG,10.99,Women,EE364229B2791D1EF9355708EFF0BA34,1
3,14157,4.64877,Accessories,Enzyme Regular Solid Army Caps-Olive W35S45D (...,MG,10.99,Women,00BD13095D06C20B11A2993CA419D16B,1
4,14273,6.50793,Accessories,Washed Canvas Ivy Cap - Black W11S64C,MG,15.99,Women,F531DC20FDE20B7ADF3A73F52B71D0AF,1


**products_df**

| Column Name              | Description                                                                                                      |
| ------------------------ | ---------------------------------------------------------------------------------------------------------------- |
| `id`                     | Unique product ID                                                                                                |
| `cost`                   | Internal cost of the product for the retailer                                                                    |
| `category`               | Product category (e.g., Accessories, Tops, Shoes)                                                                |
| `name`                   | Full name or description of the product                                                                          |
| `brand`                  | Brand or manufacturer of the product                                                                             |
| `retail_price`           | Recommended retail price for the product                                                                         |
| `department`             | Department the product belongs to (e.g., Women, Men, Kids)                                                       |
| `sku`                    | Stock Keeping Unit: unique code used to identify and track inventory                                             |
| `distribution_center_id` | Foreign key linking to the `distribution_centers` table, indicating where the product is stocked or shipped from |


In [10]:
users_df = thelook_dfs['users']
users_df.head(5)

,id,first_name,last_name,email,age,gender,state,street_address,postal_code,city,country,latitude,longitude,traffic_source,created_at,user_geom
0,72463,Russell,Lloyd,russelllloyd@example.net,61,M,Acre,14277 Brown Fork,69980-000,null,Brasil,-8.065346,-72.870949,Search,2021-02-25 06:06:00+00:00,POINT(-72.87094866 -8.065346116)
1,18973,Megan,Miles,meganmiles@example.com,16,F,Acre,31738 Briana Villages,69980-000,null,Brasil,-8.065346,-72.870949,Search,2024-11-27 05:52:00+00:00,POINT(-72.87094866 -8.065346116)
2,17109,Tyler,Alvarado,tyleralvarado@example.com,19,M,Acre,713 Ricky Estates,69980-000,null,Brasil,-8.065346,-72.870949,Organic,2023-09-21 10:57:00+00:00,POINT(-72.87094866 -8.065346116)
3,9331,Jose,Moore,josemoore@example.org,19,M,Acre,22030 Katelyn Estate Apt. 252,69980-000,null,Brasil,-8.065346,-72.870949,Search,2021-04-16 13:46:00+00:00,POINT(-72.87094866 -8.065346116)
4,80443,Lance,Newman,lancenewman@example.net,36,M,Acre,069 Dennis Trail,69980-000,null,Brasil,-8.065346,-72.870949,Search,2019-11-05 02:40:00+00:00,POINT(-72.87094866 -8.065346116)


**users_df dictionary**

  | Column Name      | Description                                                                                |
| ---------------- | ------------------------------------------------------------------------------------------ |
| `id`             | Unique identifier for the user (primary key)                                               |
| `first_name`     | User's first name                                                                          |
| `last_name`      | User's last name                                                                           |
| `email`          | User's email address                                                                       |
| `age`            | Age of the user                                                                            |
| `gender`         | Gender of the user (`M`, `F`, or other)                                                    |
| `state`          | State of residence (e.g., Acre)                                                            |
| `street_address` | Full street address of the user                                                            |
| `postal_code`    | Postal or ZIP code                                                                         |
| `city`           | City (some entries may be `null`)                                                          |
| `country`        | Country of residence (e.g., Brasil)                                                        |
| `latitude`       | Latitude coordinate of the user's address                                                  |
| `longitude`      | Longitude coordinate of the user's address                                                 |
| `traffic_source` | Original acquisition source (e.g., Facebook, Email, Search, Organic)                       |
| `created_at`     | Timestamp of when the user was created in the system                                       |
| `user_geom`      | Spatial data point combining `longitude` and `latitude` (for mapping and geospatial joins) |


### Data Cleaning & Exploration

#### orders_df

In [11]:
# number of colums
orders_df.shape

(125188, 9)

In [12]:
# Check data type of the orders_df

orders_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125188 entries, 0 to 125187
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype              
---  ------        --------------   -----              
 0   order_id      125188 non-null  Int64              
 1   user_id       125188 non-null  Int64              
 2   status        125188 non-null  object             
 3   gender        125188 non-null  object             
 4   created_at    125188 non-null  datetime64[us, UTC]
 5   returned_at   12347 non-null   datetime64[us, UTC]
 6   shipped_at    81376 non-null   datetime64[us, UTC]
 7   delivered_at  43917 non-null   datetime64[us, UTC]
 8   num_of_item   125188 non-null  Int64              
dtypes: Int64(3), datetime64[us, UTC](4), object(2)
memory usage: 9.0+ MB


In [13]:
# Check for null values

orders_df.isnull().sum()

order_id             0
user_id              0
status               0
gender               0
created_at           0
returned_at     112841
shipped_at       43812
delivered_at     81271
num_of_item          0
dtype: int64

In [14]:
orders_df['status'].unique()

array(['Cancelled', 'Complete', 'Processing', 'Returned', 'Shipped'],
      dtype=object)

In [15]:
orders_df['status'].value_counts(normalize = True)

status
Shipped       0.299222
Complete      0.252181
Processing    0.201042
Cancelled     0.148928
Returned      0.098628
Name: proportion, dtype: float64

In [16]:
# unique values and proportion of items in gender

orders_df['gender'].value_counts(normalize = True)

gender
F    0.500447
M    0.499553
Name: proportion, dtype: float64

In [17]:
orders_df.describe()

,order_id,user_id,num_of_item
count,125188.0,125188.0,125188.0
mean,62594.5,49975.070798,1.450594
std,36138.807087,28894.560164,0.807067
min,1.0,1.0,1.0
25%,31297.75,24937.5,1.0
50%,62594.5,49964.0,1.0
75%,93891.25,74968.25,2.0
max,125188.0,100000.0,4.0


**distribution_df (Distribution center)**

**Null values interpretation**

The order_df has 112122 null valued for returned_at, this indicates that 112122 were not returned, 43852 were never shipped, and 81210 has no delivery time, perhaps because they were never delivered. This could include:

- Orders that were cancelled before delivery.

- Orders that are still in transit.

- Orders that were lost or failed delivery.

**Status Value counts**

About 29.85% of orders were shipped but not yet marked as delivered, 25.14 were succesfully delivered, 19.92 % are currenly been processed, 15.11 % were cancelled before completion and 9.96 orders were delivered by returned by customers. 

**Gender**

`50.06` % of customers are Male and  `49.93` are female

**.decribe**

50% media of all orders contain 1 item and 75% of order contain 2 items and the maximum number of items ordered in a single order is 4

In [18]:
distribution_df.head()

,id,name,latitude,longitude,distribution_center_geom
0,8,Mobile AL,30.6944,-88.0431,POINT(-88.0431 30.6944)
1,9,Charleston SC,32.7833,-79.9333,POINT(-79.9333 32.7833)
2,6,Port Authority of New York/New Jersey NY/NJ,40.6340,-73.7834,POINT(-73.7834 40.634)
3,5,New Orleans LA,29.9500,-90.0667,POINT(-90.0667 29.95)
4,2,Chicago IL,41.8369,-87.6847,POINT(-87.6847 41.8369)


In [19]:
# no of rows and columns
distribution_df.shape

(10, 5)

In [20]:
# check for null values

distribution_df.isnull().sum()

id                          0
name                        0
latitude                    0
longitude                   0
distribution_center_geom    0
dtype: int64

In [21]:
distribution_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        10 non-null     Int64  
 1   name                      10 non-null     object 
 2   latitude                  10 non-null     float64
 3   longitude                 10 non-null     float64
 4   distribution_center_geom  10 non-null     object 
dtypes: Int64(1), float64(2), object(2)
memory usage: 542.0+ bytes


**events_df**

In [22]:
# number of rows in the table
events_df.shape

(2429240, 13)

In [23]:
events_df.head(5)

,id,user_id,sequence_number,session_id,created_at,ip_address,city,state,postal_code,browser,traffic_source,uri,event_type
0,2252100,<NA>,3,e3fc7136-1c43-43ab-9de7-ce2ce5caff62,2022-06-27 11:57:00+00:00,16.139.234.95,São Paulo,São Paulo,02675-031,Firefox,Organic,/cancel,cancel
1,2285223,<NA>,3,903f30d8-4491-471c-bd44-5fc65375b069,2021-10-22 19:03:00+00:00,3.205.142.27,São Paulo,São Paulo,02675-031,IE,Facebook,/cancel,cancel
2,2217907,<NA>,3,1501e6b7-dbf7-4d4c-93e4-5c36e6c7b7af,2021-08-28 04:03:00+00:00,182.41.211.27,São Paulo,São Paulo,02675-031,Other,Email,/cancel,cancel
3,1517876,<NA>,3,f6072efc-f9e5-4a60-8e06-b4a7a8ecbf26,2020-10-24 09:25:00+00:00,71.67.3.226,São Paulo,São Paulo,04941-175,Firefox,Organic,/cancel,cancel
4,1754892,<NA>,3,3ca48bf1-822d-4bbf-87c4-4e0b91ad7878,2024-11-26 07:25:00+00:00,79.186.77.31,Embu-Guaçu,São Paulo,06900-000,Safari,Email,/cancel,cancel


In [24]:
#check data types
events_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2429240 entries, 0 to 2429239
Data columns (total 13 columns):
 #   Column           Dtype              
---  ------           -----              
 0   id               Int64              
 1   user_id          Int64              
 2   sequence_number  Int64              
 3   session_id       object             
 4   created_at       datetime64[us, UTC]
 5   ip_address       object             
 6   city             object             
 7   state            object             
 8   postal_code      object             
 9   browser          object             
 10  traffic_source   object             
 11  uri              object             
 12  event_type       object             
dtypes: Int64(3), datetime64[us, UTC](1), object(9)
memory usage: 247.9+ MB


In [25]:
#check for null values

events_df.isnull().sum()

id                       0
user_id            1125021
sequence_number          0
session_id               0
created_at               0
ip_address               0
city                     0
state                    0
postal_code              0
browser                  0
traffic_source           0
uri                      0
event_type               0
dtype: int64

In [26]:
# For user who are not logged in, what kind of events did they generate and how often 

events_df[events_df['user_id'].isnull()]['event_type'].value_counts(normalize = True)

event_type
product       0.444436
department    0.222315
cart          0.222265
cancel        0.110984
Name: proportion, dtype: float64

In [27]:
events_df['event_type'].unique()

array(['cancel', 'cart', 'department', 'home', 'product', 'purchase'],
      dtype=object)

**events_df breakdown**
  
The event table likely captures website /app user behaviour such as clicks, page visists and cancellations. user_id has 1125736 null values which could be customers who visited as guests, or people who simply did not sign in. 

Each website visitor, either signed in or not performs certain actions which are 'cancel', 'cart', 'department', 'home', 'product', 'purchase'"

**Why event type is important**

These event types are valuable for:

- Funnel analysis (home → department → product → cart → purchase)

- Drop-off analysis (e.g., many reach cart but few purchase)

- Engagement segmentation (users who only browse vs. those who purchase)

- Personalization and recommendation logic

**inventoryitems_df**

In [28]:
# number of columns and rows
inventoryitems_df.shape

(490519, 12)

In [29]:
# check data type
inventoryitems_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 490519 entries, 0 to 490518
Data columns (total 12 columns):
 #   Column                          Non-Null Count   Dtype              
---  ------                          --------------   -----              
 0   id                              490519 non-null  Int64              
 1   product_id                      490519 non-null  Int64              
 2   created_at                      490519 non-null  datetime64[us, UTC]
 3   sold_at                         181597 non-null  datetime64[us, UTC]
 4   cost                            490519 non-null  float64            
 5   product_category                490519 non-null  object             
 6   product_name                    490519 non-null  object             
 7   product_brand                   490519 non-null  object             
 8   product_retail_price            490519 non-null  float64            
 9   product_department              490519 non-null  object             
 

In [30]:
# checking null value

inventoryitems_df.isnull().sum()

id                                     0
product_id                             0
created_at                             0
sold_at                           308922
cost                                   0
product_category                       0
product_name                           0
product_brand                          0
product_retail_price                   0
product_department                     0
product_sku                            0
product_distribution_center_id         0
dtype: int64

In [31]:
inventoryitems_df.head()

,id,product_id,created_at,sold_at,cost,product_category,product_name,product_brand,product_retail_price,product_department,product_sku,product_distribution_center_id
0,90209,4213,2022-04-30 10:32:09+00:00,2022-05-20 02:33:09+00:00,16.24435,Jumpsuits & Rompers,A Elan Usa Beautiful Strapless Jumpsuit w/ 3/4...,Elan USA,32.950001,Women,872DD316EB8A432CBC63F141E2D68DED,2
1,90210,4213,2021-10-01 07:20:00+00:00,NaT,16.24435,Jumpsuits & Rompers,A Elan Usa Beautiful Strapless Jumpsuit w/ 3/4...,Elan USA,32.950001,Women,872DD316EB8A432CBC63F141E2D68DED,2
2,90211,4213,2020-03-28 03:46:00+00:00,NaT,16.24435,Jumpsuits & Rompers,A Elan Usa Beautiful Strapless Jumpsuit w/ 3/4...,Elan USA,32.950001,Women,872DD316EB8A432CBC63F141E2D68DED,2
3,90212,4213,2022-04-04 08:01:00+00:00,NaT,16.24435,Jumpsuits & Rompers,A Elan Usa Beautiful Strapless Jumpsuit w/ 3/4...,Elan USA,32.950001,Women,872DD316EB8A432CBC63F141E2D68DED,2
4,146617,4213,2024-12-31 11:56:22+00:00,2025-01-12 11:32:22+00:00,16.24435,Jumpsuits & Rompers,A Elan Usa Beautiful Strapless Jumpsuit w/ 3/4...,Elan USA,32.950001,Women,872DD316EB8A432CBC63F141E2D68DED,2


**inventoryitems_df null values**

All features has 0 null values excpet sold_at with 307760 null values, this shows items that haven't been sold

**orderitems_df**

In [32]:
# numbert of columns and rows
orderitems_df.shape

(181597, 11)

In [33]:
# null values
orderitems_df.isnull().sum()

id                        0
order_id                  0
user_id                   0
product_id                0
inventory_item_id         0
status                    0
created_at                0
shipped_at            63612
delivered_at         117780
returned_at          163583
sale_price                0
dtype: int64

In [34]:
orderitems_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181597 entries, 0 to 181596
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype              
---  ------             --------------   -----              
 0   id                 181597 non-null  Int64              
 1   order_id           181597 non-null  Int64              
 2   user_id            181597 non-null  Int64              
 3   product_id         181597 non-null  Int64              
 4   inventory_item_id  181597 non-null  Int64              
 5   status             181597 non-null  object             
 6   created_at         181597 non-null  datetime64[us, UTC]
 7   shipped_at         117985 non-null  datetime64[us, UTC]
 8   delivered_at       63817 non-null   datetime64[us, UTC]
 9   returned_at        18014 non-null   datetime64[us, UTC]
 10  sale_price         181597 non-null  float64            
dtypes: Int64(5), datetime64[us, UTC](4), float64(1), object(1)
memory usage: 16.1+ MB


**orderitems_df null values**

63,457 orders have not been shipped yet or could indicate cancelled orders . 117650 indicates orders that were never delivered and 162843 shows the number of items that were never delivered. 

**products_df**

In [35]:
# number of rows and columns
products_df.shape


(29120, 9)

In [36]:
products_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29120 entries, 0 to 29119
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      29120 non-null  Int64  
 1   cost                    29120 non-null  float64
 2   category                29120 non-null  object 
 3   name                    29118 non-null  object 
 4   brand                   29096 non-null  object 
 5   retail_price            29120 non-null  float64
 6   department              29120 non-null  object 
 7   sku                     29120 non-null  object 
 8   distribution_center_id  29120 non-null  Int64  
dtypes: Int64(2), float64(2), object(5)
memory usage: 2.1+ MB


In [37]:
# check null value

products_df.isnull().sum()

id                         0
cost                       0
category                   0
name                       2
brand                     24
retail_price               0
department                 0
sku                        0
distribution_center_id     0
dtype: int64

In [38]:
# replace null brand values with unknown

products_df['brand'] = products_df['brand'].fillna('unknown')

In [39]:
# products without name

products_df[products_df['name'].isnull()]

,id,cost,category,name,brand,retail_price,department,sku,distribution_center_id
3247,12586,18.972000,Intimates,None,Josie by Natori,36.000000,Women,A7EA034186E14FB5F7B37CF664893CD2,1
5588,24455,67.335453,Outerwear & Coats,None,Tru-Spec,147.990005,Men,B290A635641F585B3DD6B95FD42DC267,2


In [40]:
# drop null values in name

products_df = products_df.dropna(subset = ['name'])

**users_df**

In [41]:
# for number of rows and columns
users_df.shape

(100000, 16)

In [42]:
users_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 16 columns):
 #   Column          Non-Null Count   Dtype              
---  ------          --------------   -----              
 0   id              100000 non-null  Int64              
 1   first_name      100000 non-null  object             
 2   last_name       100000 non-null  object             
 3   email           100000 non-null  object             
 4   age             100000 non-null  Int64              
 5   gender          100000 non-null  object             
 6   state           100000 non-null  object             
 7   street_address  100000 non-null  object             
 8   postal_code     100000 non-null  object             
 9   city            100000 non-null  object             
 10  country         100000 non-null  object             
 11  latitude        100000 non-null  float64            
 12  longitude       100000 non-null  float64            
 13  traffic_source 

In [43]:
# Null values

users_df.isnull().sum()

id                0
first_name        0
last_name         0
email             0
age               0
gender            0
state             0
street_address    0
postal_code       0
city              0
country           0
latitude          0
longitude         0
traffic_source    0
created_at        0
user_geom         0
dtype: int64

## Creating a Mastersheet 

**join order_df and users_df on user_id**

In [44]:
#check the columns in each table and drop duplicates before merging
print(f'order_df columns : {orders_df.columns}')


print(f'users_df columns : {users_df.columns}')

order_df columns : Index(['order_id', 'user_id', 'status', 'gender', 'created_at', 'returned_at',
       'shipped_at', 'delivered_at', 'num_of_item'],
      dtype='object')
users_df columns : Index(['id', 'first_name', 'last_name', 'email', 'age', 'gender', 'state',
       'street_address', 'postal_code', 'city', 'country', 'latitude',
       'longitude', 'traffic_source', 'created_at', 'user_geom'],
      dtype='object')


In [45]:
# a function to identify duplicates columns 

def duplicates(x, y):
    ''' These function check the columns in 
    two tables and identify over lapping columns
    '''
    return [c for c in x.columns if c in y.columns]

In [46]:
# check the columns in users_df and orders_df 

duplicates(orders_df, users_df)

['gender', 'created_at']

#### Over lapping columns.

'gender', 'created_at' are both in orders_df and users_df tables. The gender column will be dropped from the order_df table.   The created_at will be renamed for both columns. users_df['created_at'] indicates the account creation date while orders_df['created_at] indicates order creation date. 
Both columns serves different purposes.  

`users_df['created_at']` will be renamed as `user_created_at`
`orders['created_at']` will be renamed as `order_created_at`

In [56]:
# drop gender column

orders_df = orders_df.drop(columns = 'gender')

In [58]:
# rename created_at columns
orders_df = orders_df.rename(columns = {'created_at': 'order_created_at'})
users_df = users_df.rename(columns = {'created_at': 'user_created_at'})

In [59]:
#merge the two tables
user_orders = orders_df.merge(users_df, left_on = 'user_id', right_on = 'id', how = 'inner' )
user_orders.head()

,order_id,user_id,status,order_created_at,returned_at,shipped_at,delivered_at,num_of_item,id,first_name,...,state,street_address,postal_code,city,country,latitude,longitude,traffic_source,user_created_at,user_geom
0,8,5,Cancelled,2025-03-04 06:46:00+00:00,NaT,NaT,NaT,1,5,Kathleen,...,Maranhão,572 Kline Cape,65043,São Luís,Brasil,-2.556385,-44.259429,Search,2020-12-28 06:46:00+00:00,POINT(-44.25942906 -2.556385377)
1,9,6,Cancelled,2025-06-03 03:06:00+00:00,NaT,NaT,NaT,1,6,Tammie,...,Arizona,99906 Kristy Grove,85350,San Luis,United States,32.552220,-114.703520,Search,2019-11-19 03:06:00+00:00,POINT(-114.7035205 32.55222047)
2,16,15,Cancelled,2022-04-04 11:48:00+00:00,NaT,NaT,NaT,1,15,Michaela,...,Xinjiang Uygur Autonomous Region,336 Andre Spurs Suite 616,830000,Liuzhou,China,43.793980,87.579124,Search,2020-05-09 11:48:00+00:00,POINT(87.57912445 43.79397952)
3,44,39,Cancelled,2025-08-11 05:15:00+00:00,NaT,NaT,NaT,1,39,Mary,...,Hebei,130 Michael Drive,73000,Handan,China,38.533958,114.933484,Search,2023-07-15 05:15:00+00:00,POINT(114.9334838 38.53395821)
4,67,53,Cancelled,2025-08-13 01:00:00+00:00,NaT,NaT,NaT,1,53,Melissa,...,Gyeonggi-do,21694 Todd Rapid Apt. 059,445-938,Hwaseong City,South Korea,37.103029,126.917488,Search,2025-02-08 01:00:00+00:00,POINT(126.9174876 37.10302896)


In [68]:
# user_id and id represents the same features... id will be dropped

user_orders = user_orders.drop(columns = 'id')

In [92]:
# Write a function to check the quality of merged table 

def quality(df):
    ''' check shape null values and duplicated rows
    after merging tabl;es
    '''
    print('Shape of merged tables :', df.shape)
    print( '\nMissing values : \n', df.isnull().sum())
    print('\nNumber of Duplicate Rows:', df.duplicated().sum())

In [94]:
# check for null, duplicates and shape of user_orders

quality(user_orders)

Shape of merged tables : (125188, 19)

Missing values : 
 order_id            0
user_id             0
order_created_at    0
num_of_item         0
first_name          0
last_name           0
email               0
age                 0
gender              0
state               0
street_address      0
postal_code         0
city                0
country             0
latitude            0
longitude           0
traffic_source      0
user_created_at     0
user_geom           0
dtype: int64

Number of Duplicate Rows: 0


#### Merge orderitems_df with user_orders

In [72]:
# using the function, check for overlapping columns
duplicates(orderitems_df, user_orders)

['order_id', 'user_id', 'status', 'shipped_at', 'delivered_at', 'returned_at']

#### House keeping

`orderitems_df` and `user_orders` will be merged on `order_id`, hence, `user_id` will be dropped from orderitems_df. Status, shipped_at, delivered_at, returned_at in order_items_df captures unique information  at the item level in an order. 
Hence, Status, 'shipped_at', 'delivered_at', 'returned_at in user_orders will be dropped too, this is because the analysis is focused on item_level and not at the order level.

In [78]:
# dropping over lapping columns in user_orders

user_orders = user_orders.drop(columns = ['status', 'shipped_at', 'delivered_at', 'returned_at'])

# dropping user_id in orderitems_df

orderitems_df = orderitems_df.drop(columns = 'user_id')

In [73]:
# rename id in orderitems_df as this shows the id of each item in an order
orderitems_df = orderitems_df.rename(columns = {'id' : 'orderitem_id' })

In [81]:
# House keeping 
''' check for over lapping columns
'''

duplicates(orderitems_df, user_orders)

['order_id']

In [85]:
# merge orderitems_d & user_orders

complete_orders_df = orderitems_df.merge(user_orders, on = 'order_id', how = 'inner')

In [87]:
#confirm shape
complete_orders_df.shape

(181597, 28)

In [91]:
quality(complete_orders_df)

Shape of merged tables : (181597, 28)

Missing values : 
 orderitem_id              0
order_id                  0
product_id                0
inventory_item_id         0
status                    0
created_at                0
shipped_at            63612
delivered_at         117780
returned_at          163583
sale_price                0
user_id                   0
order_created_at          0
num_of_item               0
first_name                0
last_name                 0
email                     0
age                       0
gender                    0
state                     0
street_address            0
postal_code               0
city                      0
country                   0
latitude                  0
longitude                 0
traffic_source            0
user_created_at           0
user_geom                 0
dtype: int64

Number of Duplicate Rows: 0


#### Merge inventoryitems_df and complete_orders_df

In [106]:
inventoryitems_df.head(5)

,id,product_id,inventorycreated_at,sold_at,cost,product_category,product_name,product_brand,product_retail_price,product_department,product_sku,product_distribution_center_id
0,90209,4213,2022-04-30 10:32:09+00:00,2022-05-20 02:33:09+00:00,16.24435,Jumpsuits & Rompers,A Elan Usa Beautiful Strapless Jumpsuit w/ 3/4...,Elan USA,32.950001,Women,872DD316EB8A432CBC63F141E2D68DED,2
1,90210,4213,2021-10-01 07:20:00+00:00,NaT,16.24435,Jumpsuits & Rompers,A Elan Usa Beautiful Strapless Jumpsuit w/ 3/4...,Elan USA,32.950001,Women,872DD316EB8A432CBC63F141E2D68DED,2
2,90211,4213,2020-03-28 03:46:00+00:00,NaT,16.24435,Jumpsuits & Rompers,A Elan Usa Beautiful Strapless Jumpsuit w/ 3/4...,Elan USA,32.950001,Women,872DD316EB8A432CBC63F141E2D68DED,2
3,90212,4213,2022-04-04 08:01:00+00:00,NaT,16.24435,Jumpsuits & Rompers,A Elan Usa Beautiful Strapless Jumpsuit w/ 3/4...,Elan USA,32.950001,Women,872DD316EB8A432CBC63F141E2D68DED,2
4,146617,4213,2024-12-31 11:56:22+00:00,2025-01-12 11:32:22+00:00,16.24435,Jumpsuits & Rompers,A Elan Usa Beautiful Strapless Jumpsuit w/ 3/4...,Elan USA,32.950001,Women,872DD316EB8A432CBC63F141E2D68DED,2


In [107]:
# check for over lapping columns
duplicates(inventoryitems_df, complete_orders_df)

['product_id']

**Overlapping columns**

The column created_at will be renamed as inventory_created_at as this is the date/time when the inventory for the product was created and inventoryitems_df, complete_orders_df will be merged on inventory_item_id. 
The over lapping product_id will be dropped for inventoryitems_df

In [120]:
# Renaming created_at

inventoryitems_df = inventoryitems_df.rename(columns = {'created_at' : 'inventorycreated_at'})

# drop inventoryitems_df product_id

inventoryitems_df = inventoryitems_df.drop(columns = ['product_id'])


In [121]:
# merge 
with_inventory_df = complete_orders_df.merge(
    inventoryitems_df, left_on = 'product_id', 
    right_on = 'id', 
    how = 'inner' 
)

In [122]:
# drop id from the merged table
with_inventory_df = with_inventory_df.drop(columns = ['id'])

In [123]:
# sanity check

quality(with_inventory_df)

Shape of merged tables : (181597, 38)

Missing values : 
 orderitem_id                           0
order_id                               0
product_id                             0
inventory_item_id                      0
status                                 0
created_at                             0
shipped_at                         63612
delivered_at                      117780
returned_at                       163583
sale_price                             0
user_id                                0
order_created_at                       0
num_of_item                            0
first_name                             0
last_name                              0
email                                  0
age                                    0
gender                                 0
state                                  0
street_address                         0
postal_code                            0
city                                   0
country                                0

#### Add products

In [114]:
products_df.head()

,id,cost,category,name,brand,retail_price,department,sku,distribution_center_id
0,13842,2.51875,Accessories,Low Profile Dyed Cotton Twill Cap - Navy W39S55D,MG,6.25,Women,EBD58B8A3F1D72F4206201DA62FB1204,1
1,13928,2.33835,Accessories,Low Profile Dyed Cotton Twill Cap - Putty W39S55D,MG,5.95,Women,2EAC42424D12436BDD6A5B8A88480CC3,1
2,14115,4.87956,Accessories,Enzyme Regular Solid Army Caps-Black W35S45D,MG,10.99,Women,EE364229B2791D1EF9355708EFF0BA34,1
3,14157,4.64877,Accessories,Enzyme Regular Solid Army Caps-Olive W35S45D (...,MG,10.99,Women,00BD13095D06C20B11A2993CA419D16B,1
4,14273,6.50793,Accessories,Washed Canvas Ivy Cap - Black W11S64C,MG,15.99,Women,F531DC20FDE20B7ADF3A73F52B71D0AF,1


In [124]:
# check for over lapping columns

duplicates(products_df, with_inventory_df)

['cost']

products_df['cost'] This is the product level cost
inventoryitems_df['cost'] : This is the unit cost of the product to the business. If the business stocks different batches, the unity price of the bacth may be different from each other.

In [133]:
# renaming cost for the two tables
products_df = products_df.rename(columns = {'cost' : 'product_cost'})

with_inventory_df = with_inventory_df.rename(columns = {'cost' : 'inventory_cost'})

In [139]:
# merge 
merged_df = with_inventory_df.merge(
    products_df, left_on = 'product_id', 
    right_on = 'id', 
    how = 'inner' 
)

In [140]:
quality(merged_df)

Shape of merged tables : (181586, 47)

Missing values : 
 orderitem_id                           0
order_id                               0
product_id                             0
inventory_item_id                      0
status                                 0
created_at                             0
shipped_at                         63607
delivered_at                      117771
returned_at                       163573
sale_price                             0
user_id                                0
order_created_at                       0
num_of_item                            0
first_name                             0
last_name                              0
email                                  0
age                                    0
gender                                 0
state                                  0
street_address                         0
postal_code                            0
city                                   0
country                                0

In [141]:
merged_df.head(5)

,orderitem_id,order_id,product_id,inventory_item_id,status,created_at,shipped_at,delivered_at,returned_at,sale_price,...,product_distribution_center_id,id,product_cost,category,name,brand,retail_price,department,sku,distribution_center_id
0,111996,77190,14235,302621,Complete,2023-05-24 20:52:01+00:00,2023-05-26 10:55:00+00:00,2023-05-29 12:40:00+00:00,NaT,0.02,...,8,14235,0.0083,Accessories,Indestructable Aluminum Aluma Wallet - RED,marshal,0.02,Women,8425BC94A44E3D1BB3C8C026B2702C00,1
1,46135,31723,14235,124693,Processing,2025-08-07 04:48:18+00:00,NaT,NaT,NaT,0.02,...,8,14235,0.0083,Accessories,Indestructable Aluminum Aluma Wallet - RED,marshal,0.02,Women,8425BC94A44E3D1BB3C8C026B2702C00,1
2,51295,35248,14235,138612,Returned,2024-02-26 09:48:25+00:00,2024-02-28 17:19:00+00:00,2024-03-03 03:36:00+00:00,2024-03-04 06:32:00+00:00,0.02,...,8,14235,0.0083,Accessories,Indestructable Aluminum Aluma Wallet - RED,marshal,0.02,Women,8425BC94A44E3D1BB3C8C026B2702C00,1
3,51986,35737,14235,140490,Shipped,2023-06-15 12:07:34+00:00,2023-06-18 06:34:00+00:00,NaT,NaT,0.02,...,8,14235,0.0083,Accessories,Indestructable Aluminum Aluma Wallet - RED,marshal,0.02,Women,8425BC94A44E3D1BB3C8C026B2702C00,1
4,95388,65821,14235,257767,Shipped,2025-02-12 00:52:53+00:00,2025-02-11 04:32:00+00:00,NaT,NaT,0.02,...,8,14235,0.0083,Accessories,Indestructable Aluminum Aluma Wallet - RED,marshal,0.02,Women,8425BC94A44E3D1BB3C8C026B2702C00,1


#### saving the merged table

In [138]:
# creating folder 'Data'
folder_name = 'Data'
try:
    os.mkdir(folder_name)
    print(f" Folder {folder_name} created successfully" )
except FileExistsError:
    print(f" Folder {folder_name} already exist" )

 Folder Data created successfully


In [143]:
# Saving the table

merged_df.to_csv('Data/merged_df.csv', index = False)